# Создание и преобразование столбцов с помощью функций

In [59]:
import pandas as pd
from IPython.display import display
melb_df = pd.read_csv('data/melb_data_ps.csv', sep=',')

В наших данных есть столбец с адресами объектов недвижимости, где очень много уникальных значений

In [60]:
# NUNIQUE() - метод позволяет узнать кол-во уникальных значений в столбце
print(melb_df['Address'].nunique())

13378


Такой признак, скорее всего, не имеет **статистической значимости**, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку

Попытаемся преобразовать адреса в более информативный для статистики тип (разделим на тип дороги - шоссе, проспект, улица и т п)

In [61]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию

In [62]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

##### Метод apply() - возвращает результат применения функции (аргумента метода) к элементу столбца

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца **apply()**. В результате получим объект Series, который положим в переменную street_types

In [63]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Функция-преобразование (lambda-функция-преобразование), которая возвращает вместо значений Avenue, Boulevard и Parade их топографические сокращения (Av, Bvd, Pde)

In [64]:
dirty_names = ['Avenue', 'Boulevard', 'Parade']
clear_names = ['Av', 'Bvd', 'Pde']
street_types = street_types.apply(lambda x: x if x not in dirty_names else clear_names[dirty_names.index(x)])

Теперь уникальных значений стало меньше (можно категоризировать данные из этого столбца)

In [65]:
print(street_types.nunique())

53


Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts

In [66]:
display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: Address, dtype: int64

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

##### Метод nlargest() - возвращает n наибольших значений из Series

Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes

In [67]:
popular_stypes = street_types.value_counts().nlargest(10).index # INDEX - извлечение столбца с названием строк (индексов)
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


##### lambda-функция

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём **lambda-функцию**, которая будет проверять, есть ли строка **x** в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [68]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other') # новый столбец в датафрейм = список_типов(вернуть элемент списка если он есть в популярных_типах иначе вернуть "другой")
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [69]:
# Результирующее число подтипов улиц в получившемся столбце
print(melb_df['StreetType'].nunique())

11


Удалим "Адрес" из нашей таблицы (теперь если что, у нас есть координаты)

In [70]:
melb_df = melb_df.drop('Address', axis=1)

##### Задачи

Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).

Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.

In [81]:
# функция
def get_weekend(weekday):
    type_of_day = 0
    if weekday == 5 or weekday == 6:
        type_of_day = 1
    return type_of_day
# применим к столбцу
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
aver_price_weekend = melb_df[melb_df['Weekend'] == 1]['Price'].mean()
display(round(aver_price_weekend,0))

# то же самое через лямбду
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(lambda x: 1 if x in [5,6] else 0)
aver_price_weekend = melb_df[melb_df['Weekend'] == 1]['Price'].mean()
display(round(aver_price_weekend,0))

1081199.0

1081199.0

Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.

Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

In [91]:
# показать ассортимент продавцов
display(melb_df['SellerG'].value_counts())
# создать список уникальных имён продавцов
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
# показать его
display(popular_sellers)
# изменить столбец со старыми продавцами, чтоб в нем были только 49 самых популярных и "другие"
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
# во сколько раз минимальная цена продажи "Нельсоном" больше минимальной цены продажи "другими"
min_comparer = (melb_df[melb_df['SellerG'] == "Nelson"]["Price"].min())/(melb_df[melb_df['SellerG'] == "other"]["Price"].min())
display(round(min_comparer,1))

Nelson           1565
Jellis           1316
other            1234
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Purplebricks       51
Eview              51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Alexkarbon         40
Nick      

Index(['Nelson', 'Jellis', 'other', 'hockingstuart', 'Barry', 'Ray',
       'Marshall', 'Buxton', 'Biggin', 'Brad', 'Woodards', 'Fletchers', 'Jas',
       'Greg', 'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary',
       'Harcourts', 'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine',
       'Williams', 'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR',
       'O'Brien', 'C21', 'Collins', 'Cayzer', 'Purplebricks', 'Eview',
       'Philip', 'Buckingham', 'Bells', 'Thomson', 'Alexkarbon', 'Nick',
       'Burnham', 'McDonald', 'Moonee'],
      dtype='object')

1.3

Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах.

In [92]:
def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    month_amount = 0
    month_words_list = ['месяц', 'месяца', 'месяцев']
    year_words_list = ['год', 'года', 'лет']
    arg_list = arg.split(' ')
    arg_list[-1]
    if arg_list[-1] in month_words_list:
        month_amount = int(arg_list[-2])
        if arg_list[-3] in year_words_list:
            month_amount += int(arg_list[-4])*12
    elif arg_list[-1] in year_words_list:
        month_amount = int(arg_list[-2])*12
    return month_amount

if __name__ == '__main__':
    experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
    experience_month = experience_col.apply(get_experience)
    print(experience_month)

0    99
1    41
2    21
3     3
4    72
dtype: int64
